# 매칭
매칭은 첫 번째 영상에 추출한 기술자 집합 $A = \begin{Bmatrix} a_{1},a_{2},\cdots,a_{m} \end{Bmatrix}$와 두 번째 영상에서 추출한 기술자 집합 $B = \begin{Bmatrix} b_{1},b_{2},\cdots,b_{n} \end{Bmatrix}$가 주어졌을 때,   
같은 물체의 같은 곳에 해당하는 $a_{i}$와 $b_{j}$ 쌍을 모두 찾는 문제이다.

물체 인식, 물체 추적, 스테레오, 카메라 캘리브레이션 등의 다양한 문제에서 매칭이 핵심 역할을 한다.

두 기술자의 거리를 계산할 때는 보통 유클리디안 거리를 사용한다.

> $d(a,b) = \sqrt{\sum_{k=1}^{d}(a_{k}-b_{k})^{2}} = \left\|a-b\right\|$

이때 기술자는 $d$차원 벡터인데 SIFT 기술자는 $d=128$이다. 두 기술자를 $a = \begin{Bmatrix} a_{1},a_{2},\cdots,a_{d} \end{Bmatrix}$와 $b = \begin{Bmatrix} b_{1},b_{2},\cdots,b_{d} \end{Bmatrix}$로 표기한다.

### 매칭 전략
세 가지 매칭 전략을 생각 할 수 있다.

가장 단순한 첫 번째 전략은 두 기술자 $a_{i}$와 $b_{j}$의 거리가 임계값보다 작으면 매칭되었다고 간주하는 고정 임계값 방법이다.

> $d(a_{i},b_{j}) < T$

* 임계값 $T$를 너무 작게 하면 아주 가까운 쌍만 조건을 만족하므로 진짜 매칭 쌍이 조건을 통과못하는 거짓 부정(false negative)이 많이 발생한다.
* 반대로 $T$를 너무 크게 하면 조금만 비슷해도 조건을 만족하므로 진짜가 아닌데 조건을 통과하는 거짓 긍정(false positive)이 많이 발생한다.
* 따라서 고정 임계값 방법에서는 임계값 $T$를 정하는 일이 매우 중요하다.

두 번째 전략은 최근접 이웃(nearest neighbor)이다.
* $a_{i}$는 $B$에서 거리가 가장 작은 $b_{j}$를 찾고 $d(a_{i},b_{j}) < T$를 만족하면 매칭 쌍으로 취한다.

세 번째 전략은 최근접 이웃 거리 비율이다.
* $a_{i}$는 $B$에서 가장 가까운 $b_{j}$와 두 번째 가까운 $b_{k}$를 찾는다. $b_{j}$와 $b_{k}$가 아래의 식을 만족하면 $a_{i}$와 $b_{j}$는 매칭 쌍이 된다.

> $d(a_{i},b_{j}) / d(a_{i},b_{k}) < T$

많은 논문이 최근접 이웃 거리 비율 전략이 가장 좋은 성능을 보인다는 실험 결과를 보고한다. SIFT도 이 전략을 사용한다.

### 매칭 성능 측정
정량적 성능 측정은 알고리즘을 개선하거나 최선의 알고리즘을 선택하기 위한 기준이며 시스템을 현장에 투입할지 결정할 때 꼭 필요하다.

아래는 색깔로 정답(GT: Ground Truth)을 표시하는데, 같은 색은 진짜 매칭 쌍이고 다른 색은 진짜 매칭 쌍이 아니다.

<img src="GT.jpg" width="1000"/>

아래와 같이 혼동 행렬(confusion matrix)를 이용하면 성능 지표를 계산할 수 있다.

<img src="confusion_matrix.jpg" width="1000"/>

아래는 성능 지표의 목록이다.
* $precision = \frac{TP}{TP+FP}$

* $recall = \frac{TP}{TP+FN}$

* $F1 = \frac{2\times precision\times recall}{precision+recall}$

* $accuracy = \frac{TP+TN}{TP+TN+FP+FN}$

특징점 매칭에서는 부정이 긍정보다 월등히 많아 정확률(accuracy)가 의미가 없는 상황이 많다.

물체 인식은 물체만 있는 모델 영상에서 추출한 특징점은 대부분 긍정이지만, 물체와 배경이 섞인 장면 영상에서 추출한 특징점 대부분은 부정이다.

이 경우 매칭 알고리즘이 매칭 쌍을 출력하지 않아도 정확률이 100% 가깝게 되므로, 정확률을 사용할 때는 주의해야 한다.

고정 임계값 방법이나 최근접 이웃 거리 비율 방법에서 사용하는 임계값 $T$에 대해서,
* $T$를 작게하면 거짓 긍정률(FPR: False Positive Rate)은 작아진다.

* $T$를 크게하면 거짓 긍정률이 커지는데, 참 긍정률(TPR: True Positive Rate)도 따라 커지는 경향이 있다.

* $FPR = \frac{FP}{TN+FP}$

* $TPR = \frac{TP}{TP+FN}$

아래 그림은 가로축을 거짓 긍정률, 세로축을 참 긍정률로 설정한 ROC(Receiver Operating Characteristic) 곡선이다.

<img src="ROC.jpg" width="300"/>

이 그림은 가상의 매칭 알고리즘으로 측정한 세 ROC 곡선을 보여주는데, 왼쪽 위 꼭지점에 가까운 $c2$가 성능이 가장 뛰어나다.

ROC 곡선의 아래쪽 면적을 뜻하는 AUC(Area Under Curve)를 측정하여 매칭 알고리즘의 성능을 판단할 수 있다.

$c0$의 AUC는 0.5이고, $c1$의 AUC는 0.5보다 크고, $c2$의 AUC는 $c1$의 AUC보다 크다.

### 빠른 매칭
컴퓨터 비전은 매칭의 정확도도 중요하지만, 실시간 처리가 요구되는 상황에서는 속도도 매우 중요하다.

대용량 데이터를 빠르게 탐색하는 자료 구조에는 대표적으로 이진 탐색 트리(binary search tree)와 해싱(hashing)이 있다.

이진 탐색 트리는 루트 노드에서 시작하여 루트보다 작으면 왼쪽, 크면 오른쪽으로 트리를 재귀적으로 뻗어나간다.

벤트리는 이진 탐색 트리를 특징점 매칭에 적합하게 개조한 $kd$ 트리를 개발했다. (트리 생성 과정은 도서 참고)

해싱에서는 해시 함수(hash function) $h$가 키 값을 키가 저장된 칸의 번호로 매핑한다.

해싱을 특징점 매칭에 활용하려면 상당한 수정이 필요하다. 특징점 매칭에서는 키가 벡터고 최근접 이웃을 찾아야 하기 때문이다.

또한 해싱은 충돌 확률을 최소화하는 전략을 사용하지만, 특징점 매칭에 활용하려면 반대로 비슷한 특징 벡터가 같은 칸에 담길 확률을 최대한 높여야 한다.

이러한 성질을 만족하는 대표적인 해싱 기법으로 위치 의존 해싱(locality-sensitive hasing)이 있다.

### FLANN과 FAISS
SIFT를 고안한 로우 교수와 제자인 무자는 $kd$ 트리의 다양한 변형을 소개하고 이들의 성능 비교 분석한 논문을 발표했다.

또한 이들을 구현한 FLANN(Fast Library for Approximate Nearest Neighbors) 라이브러리를 공개했다.

OpenCV에서는 FLANN을 이용한 함수를 제공한다.

최근 페이스북에서는 최근접 이웃을 아주 빠르게 찾는 FAISS(Facebook AI Similarity Search) 라이브러리를 공개했다.